In [ ]:
from google.colab import files
uploaded = files.upload()


Saving sales_data_clean_transformed.csv to sales_data_clean_transformed.csv


In [ ]:
import pandas as pd
import sqlite3

# Load your data
# Load your data with 'ISO-8859-1' encoding
sales_data = pd.read_csv('sales_data_clean_transformed.csv', encoding='ISO-8859-1')

# Display the first few rows to make sure it's loaded correctly
sales_data.head()


,ORDERNUMBER,QUANTITYORDERED,PRICEEACH,ORDERLINENUMBER,SALES,ORDERDATE,STATUS,QTR_ID,MONTH_ID,YEAR_ID,...,PHONE,ADDRESSLINE1,CITY,STATE,POSTALCODE,COUNTRY,TERRITORY,CONTACTLASTNAME,CONTACTFIRSTNAME,DEALSIZE
0,10107,30,95.70,2,2871.00,2003-02-24,Shipped,1,2,2003,...,2125557818,897 Long Airport Avenue,NYC,NY,10022,USA,NaN,Yu,Kwai,Small
1,10121,34,81.35,5,2765.90,2003-05-07,Shipped,2,5,2003,...,26.47.1555,59 rue de l'Abbaye,Reims,NaN,51100,France,EMEA,Henriot,Paul,Small
2,10134,41,94.74,2,3884.34,2003-07-01,Shipped,3,7,2003,...,+33 1 46 62 7555,27 rue du Colonel Pierre Avia,Paris,NaN,75508,France,EMEA,Da Cunha,Daniel,Medium
3,10145,45,83.26,6,3746.70,2003-08-25,Shipped,3,8,2003,...,6265557265,78934 Hillside Dr.,Pasadena,CA,90003,USA,NaN,Young,Julie,Medium
4,10159,49,100.00,14,5205.27,2003-10-10,Shipped,4,10,2003,...,6505551386,7734 Strong St.,San Francisco,CA,NaN,USA,NaN,Brown,Julie,Medium


In [ ]:
# Create a connection to a SQLite database in memory
conn = sqlite3.connect(':memory:')

# Write the data to a SQLite table
sales_data.to_sql('sales', conn, index=False)


2823

In [ ]:
# Total Sales Per Year
query_1 = """
SELECT YEAR_ID, SUM(SALES) as Total_Sales
FROM sales
GROUP BY YEAR_ID
ORDER BY YEAR_ID;
"""
df_1 = pd.read_sql_query(query_1, conn)

# Format 'Total_Sales' as dollars
df_1['Total_Sales'] = df_1['Total_Sales'].map('${:,.2f}'.format)

df_1.head()


,YEAR_ID,Total_Sales
0,2003,"$3,516,979.54"
1,2004,"$4,724,162.60"
2,2005,"$1,791,486.71"


In [ ]:
# Customers Who Made The Largest Purchases
query_2 = """
SELECT CUSTOMERNAME, SUM(SALES) as Total_Sales
FROM sales
GROUP BY CUSTOMERNAME
ORDER BY Total_Sales DESC;
"""
df_2 = pd.read_sql_query(query_2, conn)

# Format 'Total_Sales' as dollars
df_2['Total_Sales'] = df_2['Total_Sales'].map('${:,.2f}'.format)

df_2.head()

,CUSTOMERNAME,Total_Sales
0,Euro Shopping Channel,"$912,294.11"
1,Mini Gifts Distributors Ltd.,"$654,858.06"
2,"Australian Collectors, Co.","$200,995.41"
3,Muscle Machine Inc,"$197,736.94"
4,La Rochelle Gifts,"$180,124.90"


In [ ]:
# Average Sale Price Per Order
query_3 = """
SELECT ORDERNUMBER, AVG(PRICEEACH) as Average_Sale_Price
FROM sales
GROUP BY ORDERNUMBER;
"""
df_3 = pd.read_sql_query(query_3, conn)

# Format 'Average_Sale_Price' as dollars
df_3['Average_Sale_Price'] = df_3['Average_Sale_Price'].map('${:,.2f}'.format)

df_3.head()

,ORDERNUMBER,Average_Sale_Price
0,10100,$72.19
1,10101,$71.24
2,10102,$75.07
3,10103,$88.60
4,10104,$81.68


In [ ]:
# Monthly Sales Trend
query_4 = """
SELECT YEAR_ID, MONTH_ID, SUM(SALES) as Total_Sales
FROM sales
GROUP BY YEAR_ID, MONTH_ID
ORDER BY YEAR_ID, MONTH_ID;
"""
df_4 = pd.read_sql_query(query_4, conn)

# Format 'Total_Sales' as dollars
df_4['Total_Sales'] = df_4['Total_Sales'].map('${:,.2f}'.format)

df_4.head()


,YEAR_ID,MONTH_ID,Total_Sales
0,2003,1,"$129,753.60"
1,2003,2,"$140,836.19"
2,2003,3,"$174,504.90"
3,2003,4,"$201,609.55"
4,2003,5,"$192,673.11"


In [ ]:
# Top 5 Product Lines by Total Sales
query_5 = """
SELECT PRODUCTLINE, Total_Sales
FROM (
    SELECT PRODUCTLINE, SUM(SALES) as Total_Sales
    FROM sales
    GROUP BY PRODUCTLINE
)
ORDER BY Total_Sales DESC
LIMIT 5;
"""
df_5 = pd.read_sql_query(query_5, conn)

# Format 'Total_Sales' as dollars
df_5['Total_Sales'] = df_5['Total_Sales'].map('${:,.2f}'.format)

df_5.head()

,PRODUCTLINE,Total_Sales
0,Classic Cars,"$3,919,615.66"
1,Vintage Cars,"$1,903,150.84"
2,Motorcycles,"$1,166,388.34"
3,Trucks and Buses,"$1,127,789.84"
4,Planes,"$975,003.57"


In [ ]:
# Year with the Most Number of Orders
query_6 = """
SELECT YEAR_ID, COUNT(DISTINCT ORDERNUMBER) as Number_of_Orders
FROM sales
GROUP BY YEAR_ID
ORDER BY Number_of_Orders DESC
LIMIT 1;
"""
df_6 = pd.read_sql_query(query_6, conn)

df_6.head()

,YEAR_ID,Number_of_Orders
0,2004,144
